In [30]:
import findspark
findspark.init()

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, sum, input_file_name, regexp_extract, regexp_replace

In [53]:
db_url = "jdbc:oracle:thin:@//localhost:1521:ORCL"
db_properties = {
    "user": "system",
    "password": "orcl1234",
    "driver": "oracle.jdbc.OracleDriver"
}

In [33]:
spark = SparkSession.builder.appName("GitHubRepos") \
.getOrCreate()

In [34]:
df = spark.read.json(['data/Angular.json','data/Cpp.json','data/Dart.json','data/Deep-Learning.json','data/Django.json','data/Docker.json','data/ethereum.json','data/Flask.json','data/Flutter.json','data/Gatsby.json','data/Golang.json','data/Hadoop.json','data/Hadoop.json','data/Julia.json','data/Kotlin.json','data/Kubernetes.json','data/Machine-Learning.json','data/NextJS.json','data/NodeJS.json','data/OOPs.json','data/PyTorch.json','data/R.json','data/React-JS.json','data/React-Native.json','data/Scala.json','data/Scikit.json','data/serverless.json','data/Spark.json','data/Tensorflow.json','data/Threejs.json','data/Typescript.json'])

In [35]:
df = df.withColumn("file_name", regexp_extract(input_file_name(), r"([^/\\]+)$", 1))
df = df.withColumn("file_name", regexp_replace(col("file_name"), r"\.json$", ""))

In [36]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- forks: long (nullable = true)
 |-- full_name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- language: string (nullable = true)
 |-- open_issues: long (nullable = true)
 |-- repo_name: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- subscribers: long (nullable = true)
 |-- topics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- type: string (nullable = true)
 |-- username: string (nullable = true)
 |-- file_name: string (nullable = false)



In [37]:
df.show()

+---------------+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+---------+
|_corrupt_record|            created|         description|forks|           full_name|       id|        language|open_issues|           repo_name|stars|subscribers|              topics|        type|           username|file_name|
+---------------+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+---------+
|           NULL|2014-02-25 08:00:08|Apache Spark - A ...|25357|        apache/spark| 17165658|           Scala|        242|               spark|32296|       2080|[python, scala, r...|Organization|             apache|    Spark|
|           NULL|2017-08-09 19:39:59|Distributed train...| 2027|     horovod/horovod| 99

PROGRAMMING_LANG

In [38]:
df_languages = df.groupBy('language').agg(count("*").alias('NUM_OF_REPOS'))

In [39]:
df_languages.show()

+----------+------------+
|  language|NUM_OF_REPOS|
+----------+------------+
|        C#|         342|
|     Terra|           1|
|  Makefile|          38|
|       VBA|           4|
|      Cuda|          12|
|       Arc|           2|
|      LLVM|           2|
|      Rust|         167|
|JavaScript|        5318|
|      TSQL|          11|
|      Perl|          16|
|    Puppet|           7|
|    Erlang|           7|
|      NULL|        1503|
|     Jinja|          15|
|       C++|         958|
|        F#|           3|
|    Groovy|          20|
|       TeX|          44|
|     OCaml|          16|
+----------+------------+
only showing top 20 rows



In [40]:
df_prog_lang =  df_languages.filter(col("language").isNotNull())

In [41]:
df_prog_lang.show()

+----------+------------+
|  language|NUM_OF_REPOS|
+----------+------------+
|        C#|         342|
|     Terra|           1|
|  Makefile|          38|
|       VBA|           4|
|      Cuda|          12|
|       Arc|           2|
|      LLVM|           2|
|      Rust|         167|
|JavaScript|        5318|
|      TSQL|          11|
|      Perl|          16|
|    Puppet|           7|
|    Erlang|           7|
|     Jinja|          15|
|       C++|         958|
|        F#|           3|
|    Groovy|          20|
|       TeX|          44|
|     OCaml|          16|
|      MQL5|           1|
+----------+------------+
only showing top 20 rows



In [42]:
df_prog_lang.write.jdbc(url=db_url, table="PROGRAMMING_LANG", mode="overwrite", properties=db_properties)

ORGNIZATION_TYPE

In [43]:
df_org_type = df.filter(col("type")=="Organization")

In [44]:
df_org_type.show()

+---------------+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+---------+
|_corrupt_record|            created|         description|forks|           full_name|       id|        language|open_issues|           repo_name|stars|subscribers|              topics|        type|           username|file_name|
+---------------+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+---------+
|           NULL|2014-02-25 08:00:08|Apache Spark - A ...|25357|        apache/spark| 17165658|           Scala|        242|               spark|32296|       2080|[python, scala, r...|Organization|             apache|    Spark|
|           NULL|2017-08-09 19:39:59|Distributed train...| 2027|     horovod/horovod| 99

In [45]:
df_org_stars = df_org_type.groupBy(col("username").alias("ORGANIZATION")).agg(sum(col("stars")).alias("TOTAL_STARS"))

In [46]:
df_org_stars.show()

+------------------+-----------+
|      ORGANIZATION|TOTAL_STARS|
+------------------+-----------+
|              didi|       5144|
|          deepmipt|      16875|
|          blei-lab|      14079|
|            GPflow|       4786|
|          twosigma|       6150|
|       intel-spark|         91|
|     Azure-Samples|       1398|
|            seznam|        162|
|            kserve|       6946|
|           SharpAI|        858|
|          Qihoo360|      20409|
|          snowlift|         52|
|          floydhub|      15620|
|         jolibrain|      11656|
|    adobe-research|        367|
|music-of-the-ainur|         22|
|           Alluxio|      22050|
|            cdapio|        162|
|    USCDataScience|        390|
|            cerner|        128|
+------------------+-----------+
only showing top 20 rows



In [47]:
df_org_stars.write.jdbc(url=db_url, table="ORGNIZATION_TYPE", mode="overwrite", properties=db_properties)

SEARCH_TERMS_RELEVENCE

In [48]:
df_agg = df.groupBy("file_name").agg(
    sum("forks").alias("total_forks"),
    sum("subscribers").alias("total_subscribers"),
    sum("stars").alias("total_stars")
)

In [49]:
df_relevance = df_agg.select(
    col("file_name").alias("search_term"), 
    (1.5 * col("total_forks") + 1.32 * col("total_subscribers") + 1.04 * col("total_stars")).alias("relevance_score")
)

In [50]:
df_relevance.show()

+------------+------------------+
| search_term|   relevance_score|
+------------+------------------+
|       Spark|295286.16000000003|
|  Tensorflow|        3918987.26|
|      Hadoop|         1168031.8|
|      Kotlin|1936763.9000000001|
|    React-JS|         870068.36|
|         Cpp|4377811.4399999995|
|    ethereum|        1157287.26|
|      Docker|4403817.5600000005|
|       Flask| 884822.1200000001|
|     Flutter|              NULL|
|       Scala|1287463.6800000002|
|     PyTorch|        3357329.08|
|React-Native|        3097924.46|
|  Typescript|        4909697.76|
|           R|     1.187633824E7|
|  Kubernetes|3540183.0200000005|
|      NodeJS|        4331014.68|
|      Scikit|         897061.24|
|      Django|1488257.1600000001|
|  serverless|        1312038.98|
+------------+------------------+
only showing top 20 rows



In [52]:
df_relevance.write.jdbc(url=db_url, table="SEARCH_TERMS_RELEVENCE", mode="overwrite", properties=db_properties)